# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Curtis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [3]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [12]:
# Fetch the Bitcoin news articles - top 20 by populatrity 
btc_headlines = newsapi.get_everything(q="bitcoin", language="en", sort_by='popularity')

In [13]:
print(f"Total articles about btc: {btc_headlines['totalResults']}")

Total articles about btc: 8519


In [20]:
# Fetch the Ethereum news articles - - top 20 by populatrity 
eth_headlines = newsapi.get_everything(q="etherium", language="en", sort_by='popularity')

In [15]:
print(f"Total articles about eth: {eth_headlines['totalResults']}")

Total articles about eth: 27


In [16]:
# Create the Bitcoin sentiment scores DataFrame


btc_sentiments = []

for article in btc_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2021-11-10,"With crypto rewards cards, a small percentage ...",0.8176,0.221,0.000,0.779
1,2021-11-05,A similar hoax earlier this year tied Walmart ...,-0.2732,0.000,0.063,0.937
2,2021-11-04,Image: Cash App\r\n\n \n\n Square has announce...,0.4767,0.075,0.000,0.925
3,2021-10-10,"Specifically, why did someone make a massive p...",0.5461,0.121,0.000,0.879
4,2021-10-16,A new cross-chain bridge is currently connecte...,0.0000,0.000,0.000,1.000


In [17]:
# Create the Ethereum sentiment scores DataFrame
eth_df = pd.DataFrame.from_dict(eth_headlines["articles"])



eth_sentiments = []

for article in eth_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()


,date,text,compound,positive,negative,neutral
0,2021-11-01,"Angelina Lee is an expert in JavaScript, TypeS...",-0.2960,0.000,0.061,0.939
1,2021-10-20,AppleInsider is supported by its audience and ...,0.4588,0.121,0.000,0.879
2,2021-11-01,"Just last year, the four thirty-somethings beh...",-0.2732,0.000,0.062,0.938
3,2021-11-05,When the largest Islamic organization in Indon...,0.1280,0.049,0.000,0.951
4,2021-10-28,Never bet against American ingenuity and innov...,-0.1531,0.075,0.092,0.833


In [18]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.160730,0.047850,0.009050,0.943100
std,0.353919,0.070065,0.029249,0.069663
min,-0.648600,0.000000,0.000000,0.779000
25%,0.000000,0.000000,0.000000,0.902250
50%,0.000000,0.000000,0.000000,0.972000
75%,0.463275,0.088500,0.000000,1.000000
max,0.817600,0.221000,0.118000,1.000000


In [19]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,0.272050,0.086100,0.026000,0.887900
std,0.317489,0.064157,0.037469,0.079318
min,-0.296000,0.000000,0.000000,0.712000
25%,0.000000,0.036750,0.000000,0.836750
50%,0.329100,0.082500,0.000000,0.896500
75%,0.490575,0.133500,0.061250,0.938250
max,0.735100,0.222000,0.098000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin (BTC) had the highest mean positive score.

Q: Which coin had the highest compound score?

A: Bitcoin (BTC) had the highest compound score.

Q. Which coin had the highest positive score?

A: Etherium (ETH) had the highest postive score. 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [21]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [22]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [37]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
   
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return output

In [48]:
# Create a new tokens column for Bitcoin
btc_tokens = []

for text in btc_df['text']:
    tokens = set(tokenizer(text),
    btc_tokens.append(tokens)
                 



SyntaxError: unexpected EOF while parsing (Temp/ipykernel_19764/610390994.py, line 8)

In [30]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [17]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [20]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [21]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [22]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [23]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [24]:
import spacy
from spacy import displacy

In [25]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [27]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [28]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [29]:
# Render the visualization
# YOUR CODE HERE!

In [30]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [32]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [33]:
# Render the visualization
# YOUR CODE HERE!

In [34]:
# List all Entities
# YOUR CODE HERE!

---